In [149]:
import numpy as np
from numpy import setdiff1d as sd
from numpy import intersect1d as inter
import json

In [230]:
in_filename = "instances/huge.json"

In [231]:
with open(in_filename, 'rb') as f:
    inst = json.load(f)

In [232]:
def parser_jobs(jobs):
    parsed = []
    for job in jobs:
        j  = job['job']
        Sj = job['sequence']
        rj = job['release_date']
        dj = job['due_date']
        wj = job['weight']
        parsed.append((j, Sj, rj, dj, wj))
    return parsed

def parser_tasks(tasks):
    parsed = []
    for task in tasks:
        i = task['task']
        pi = task['processing_time']
        Mi = parser_machines(task['machines'])
        parsed.append((i, pi, Mi))
    return parsed

def parser_machines(machines):
    parsed = []
    for machine in machines:
        m = machine['machine']
        Oim = machine['operators']
        parsed.append((m, Oim))
    return parsed

def parser_inst(inst):
    J = inst['parameters']['size']['nb_jobs']
    I = inst['parameters']['size']['nb_tasks']
    M = inst['parameters']['size']['nb_machines']
    O = inst['parameters']['size']['nb_operators']
    a = inst['parameters']['costs']['unit_penalty']
    b = inst['parameters']['costs']['tardiness']
    jobs = parser_jobs(inst['jobs'])
    tasks = parser_tasks(inst['tasks'])
    return (J, I, M, O, a, b, jobs, tasks)

In [233]:
(J, I, M, O, a, b, jobs, tasks) = parser_inst(inst)

In [234]:
print(f"J={J}\nI={I}\nM={M}\nO={a}\na={a}\nb={b}")

J=200
I=1000
M=50
O=6
a=6
b=1


In [235]:
running = []
done = []
busy_operators = []
busy_machines = []

T = 0
while len(done) < I:
    for j in jobs:
        if T >= j[2]: # it is past the release date of this job
            for (t_idx, task_id) in enumerate(j[1]): # loop over tasks for this job
                t = tasks[task_id-1] # get full task for this task id
                for m in t[2]: # look for a machine to start the task on
                    m_id = m[0] # get this machine's id
                    for op_id in m[1]: # look for an operator to operate machine
                        if (not op_id in busy_operators) \
                        and (not m_id in busy_machines) \
                        and (not task_id in [_[0] for _ in running]) \
                        and (not task_id in [_[0] for _ in done]) \
                        and inter(j[1][:t_idx], [_[0] for _ in running]).size == 0 \
                        and np.isin(j[1][:t_idx], [_[0] for _ in done]).all(): # all prior tasks done
                            running.append((task_id, T, m_id, op_id))
                            busy_operators.append(op_id)
                            busy_machines.append(m_id)
    for running_task in running:
        t_id, start_time, m_id, op_id = running_task
        processing_time = tasks[t_id-1][1]
        if (T - start_time >= processing_time):
            running = [_ for _ in running if _[0] != t_id]
            busy_operators.remove(op_id)
            busy_machines.remove(m_id)
            done.append((t_id, start_time, m_id, op_id))
        
    T += 1 # time flows
    if T % 10 == 0:
        print(T)

10
20
30
40
50
60
70
80
90
100


In [236]:
def jsonify(done):
    res = []
    for task in done:
        sub_res = {}
        sub_res['task'], sub_res['start'], sub_res['machine'], sub_res['operator'] = task
        res.append(sub_res)
    return res

In [237]:
out_filename = "KIRO-huge.json"

In [238]:
with open(out_filename, 'w') as f:
    json.dump(jsonify(done), f)